In [ ]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


In [ ]:
# imports 
import torch
import pandas as pd

from src.baseline.vocabulary import Vocabulary
from src.baseline.data_loader import get_loader, get_mean, get_std
from src.baseline.coa_model import get_new_model, init_testing_model,test_model,test_rand_image

In [ ]:
# 30 minutes to create those, as it's baseline, i ran it several times and it's the same
vocab = Vocabulary(freq_threshold=1)
vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'moline': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33}
vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11'}

# vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'molxine': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33}
# vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11'}


## Evaluate a vanilla model on the real data

In [33]:
REAL_DATASET='/home/salnabulsi/coat-of-arms/data/new/'
data_location = REAL_DATASET + '/resized'
# REAL_DATASET='/Users/salnabulsi/tub/coat-of-arms/data/cropped_coas/out'
# data_location = REAL_DATASET + '/images/resized'

test_annotation_psumsq_file  = REAL_DATASET + '/test_real_captions_psumsq.txt'
# test_annotation_psumsq_file  = REAL_DATASET + 'full-test_captions_psumsq.txt'

In [81]:
# Hyperparams
embed_size=300
vocab_size = 34
attention_dim=256
encoder_dim=512 #2048 
decoder_dim=512
learning_rate = 0.0009
drop_prob=0.5
ignored_idx = 0
batch_size = 32
NUM_WORKER = 2
freq_threshold = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }


## Get size of the model

In [82]:
# from torchinfo import summary
# summary(model)
# Rest net 50

In [83]:
# from torchinfo import summary
# summary(model)
# Rest net 18


## Init model and run the tests

In [84]:
# mean=0.3272075951099396
# std=0.3805903494358063
#09-baseline-model-test

# resized data - single and plural
# mean = 0.0118
# std = 0.0892

# # resized data - single 
# mean = 0.0115
# std = 0.0873


# mean,std = (torch.tensor(0.28803130984306335), torch.tensor(0.3481476306915283))
# print(f'Using already calculated mean and std in generated-data-api-single dataset, the mean={mean} and std={mean}')


In [85]:
mean,std = (torch.tensor(0.5218885540962219), torch.tensor(0.32714062929153445))
print(f'Using already calculated mean and std in new real dataset, the mean={mean} and std={mean}')


Using already calculated mean and std in new real dataset, the mean=0.5218885540962219 and std=0.5218885540962219


In [86]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
device

In [ ]:

import torch
import random
import numpy as np
seed = 1234


random.seed(seed)     # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
test_loader, test_dataset = init_testing_model(test_annotation_psumsq_file, data_location, 
                                               mean, std,
                                               NUM_WORKER,vocab, batch_size, 
                                               device, pin_memory=False,
                                               img_h=634, img_w=621)



In [80]:

pretrained = True
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)
test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs11', True) 


No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this labe

No shield color found in this label: "3 9 3 7 8 7 7 7 8 7 7 7 8 7 7 7 8 7 7"
No shield color found in this label: "3 9 3 7 8 7 7 7 8 7 7 7 8 7 7 7 8 7 7"
No shield color found in this label: "3 9 3 7 8 7 7 7 8 7 7 7 8 7 7 7 8 7 7"
No shield color found in this label: "3 9 3 7 8 7 7 7 8 7 7 7 8 7 7 7 8 7 7"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 li

No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
Test Accuracy ALL (in progress): 15.0%
Test Accuracy Charge-Mod only (in progress): 9.0%
Test Accuracy Charge color (in progress): 5.0%
Test Accuracy Shield color (in progress): 23.0%
No shield color found in this label: "8 7 7 8 7 8 7 7 8 7 8 7 7 8 7 8 7 7 8"
No shield color found in this label: "8 7 7 8 7 8 7 7 8 7 8 7 7 8 7 8 7 7 8

No shield color found in this label: "guard"
Caption Class - exception in label guard, ['guard']
No shield color found in this label: "guard"
Caption Class - exception in label guard, ['guard']
No shield color found in this label: "guard"
Caption Class - exception in label guard, ['guard']
No shield color found in this label: "guard"
Caption Class - exception in label guard, ['guard']
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagle

No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
ValueError (not raised): label "3 9 3 <SOS> 9 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles" cannot be parsed. The chunk

No shield color found in this label: "9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "9 3 9 lions eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
No shield color found in this label: "cross"
No shield color found in this label: "3 9 3 9 10 eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles eagles"
No shield color found in this label: "3 9 3 9 10

([3.558403730392456,
  3.5565783977508545,
  3.5564675331115723,
  3.5575146675109863,
  3.555248737335205],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.0,
  0.33,
  0.33,
  0.0,
  0.17,
  0.33,
  0.0,
  0.0,
  0.17,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.0,
  0.17,
  0.0,
  0.0,
  0.33,
  0.33,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.33,
  0.0,
  0.33,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.33,
  0.33,
  0.33,
  0.0,
  0.17,
  0.33,
  0.33,
  0.0,
  0.0,
  0.0,
  0.5,
  0.33,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.5,
  0.0,
  0.0,
  0.67,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.0,
  0.33,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.33,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,


In [ ]:
pretrained = False
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)
test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs11', True) 


In [ ]:
summary(model)

In [ ]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)

## Evaluate a vanilla model on the synthetic test data - single-and-plural-full


In [17]:
# data_location =  '../baseline-gen-data/large'
data_location =  '/home/space/datasets/COA/generated-data-api-single-and-plural-full/'
# root_folder_images = data_location + '/images'
root_folder_images = data_location + '/res_images'
test_psumsq_file = data_location + '/test_captions_psumsq.txt'
df = pd.read_csv(test_psumsq_file)

print("There are {} test images".format(len(df)))

There are 1440 test images


In [18]:
#Hyperparams
embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=512 #2048
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0
batch_size = 125
NUM_WORKER = 2
freq_threshold = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }


In [19]:
test_loader, test_dataset = init_testing_model(test_psumsq_file, root_folder_images, 
                                               mean, std,
                                               NUM_WORKER,vocab,batch_size, 
                                               device, pin_memory=False,
                                               img_h=500, img_w=500)


In [20]:
import torch
import random
import numpy as np
seed = 1234
random.seed(seed)     # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [21]:
pretrained = False
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)

test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', real_data=False) 

ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles e <UNK> e <UNK> e e e e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles e <UNK> e <UNK> e e e e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles e <UNK> e <UNK> e e e e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label s a 9 patonce, ['patonce']
ValueError (not raised): label "eagles eagles eagles <UNK> e <UNK> e e <UNK> e e e e e e e e e o" cannot be parsed. The chunk "<UNK

ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK> e eagles e <UNK> e e e e e 7 e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK> e eagles e <UNK> e e e e e 7 e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK> e eagles e <UNK> e e e e e 7 e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label g v 4 moline, ['moline']
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles e eagles e e <UNK> e e e e patonce" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles e eagles e e <UNK> e e e e patonce" cannot be parsed. 

Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles <UNK> e e <UNK> e e e o e o e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles <UNK> e e <UNK> e e e o e o e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles <UNK> e e <UNK> e e e o e o e 7" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
ValueError (not raised): label "eagles eagles eagles e eagles <UNK> e eagles e eagles e eagles e eagles e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e eagles <UNK> e eagles eagles eagles eagles e eagles e eagles 

Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label a o 6 patonce, ['patonce']
ValueError (not raised): label "eagles eagles e eagles <UNK> e eagles e eagles e <UNK> e e o e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles e eagles <UNK> e eagles e eagles e <UNK> e e o e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
ValueError (not raised): label "eagles eagles eagles <UNK> e <UNK> e <UNK> e e <UNK> e e e e e e 7 e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles <UNK> e <UNK> e <UNK> e e <UNK> e e e e e e 7 e" cannot be parsed. The chunk "<UNK>" cannot 

ValueError (not raised): label "rampant eagle eagles o e e e <UNK> e o e o e o e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e eagles <UNK> e eagles eagles eagles eagles eagles e <UNK> e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e eagles <UNK> e eagles eagles eagles eagles eagles e <UNK> e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 6 e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label o s 2 moline, ['moline']
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label a s 2 moline & border, ['moline']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 e e e

...

Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label s s 9 moline, ['moline']
Caption Class - exception in label rampant 6 6 e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 6 e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label o a 10 patonce, ['patonce']
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles e eagles eagles e eagles e eagles e patonce e patonce e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label eagles eagles eagles e <UNK> e eagles e eagles eagles e eagles e eagles e patonce e patonce e, ['', '', '', '', '', '', '', 'e', 'e']
Caption Class - exception in label g v 11 patonce & border, ['patonce']
ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK

Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label a v 11 patonce, ['patonce']
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label a o 7 moline, ['moline']
ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK> e eagles eagles e eagles e eagles e e o e o" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles <UNK> e eagles <UNK> e eagles eagles e eagles e eagles e e o e o" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles e <UNK> e <UNK> e eagles e eagles e <UNK> e e e e e e e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles e <UNK> e <UNK> e eagles e eagles e <UNK> e e e e e e e" cannot be parsed. The chunk "<UNK>" cannot

Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
Caption Class - exception in label g b 8 patonce & border, ['patonce']
Caption Class - exception in label rampant 6 e e e e e e e e e e e e e e e e e, ['rampant']
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles eagles eagles eagles eagles eagles eagles e eagles e eagles e eagles" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "rampant eagle eagles e e <UNK> e e e o e o e o e o e o e" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "eagles eagles eagles e <UNK> e eagles eagles eagles eagles eagles eagles e eagles e eagles e e o" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['ramp

Caption Class - exception in label rampant 6 v e e e e e e e e e e e e e e e e, ['rampant']
Test Accuracy (in progress): 3.0%
Test Accuracy (Overall): 3.0%
Test Loss (final):  3.67


([3.653839349746704,
  3.657409191131592,
  3.670870065689087,
  3.6717991828918457,
  3.6773712635040283,
  3.6729557514190674,
  3.6682517528533936,
  3.6748311519622803,
  3.668003797531128,
  3.677107810974121,
  3.671607494354248,
  3.6800808906555176],
 [0.165,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.25,
  0.0,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.25,
  0.25,
  0.0,
  0.0,
  0.25,
  0.0,
  0.0,
  0.25,
  0.25,
  0.25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [22]:
pretrained = True
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)

test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', real_data=False) 

No shield color found in this label: "1 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <

No shield color found in this label: "1 2 rampant 9 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
Caption Class - exception in label 1 2 rampant 9 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6, ['rampant']
ValueError (not raised): label "1 1 4 8 4 4 8 4 8 <SOS> 4 & 10 <SOS> moline s 10 8 10" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 4 4 8 4 8 <SOS> 4 & 10 <SOS> moline s 10 8 10" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
Caption Class - exception in label 1 1 4 8 4 4 8 4 8 <SOS> 4 & 10 <SOS> moline s 10 8 10, ['10']
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 

ValueError (not raised): label "1 1 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.


No shield color found in this label: "11 11 8 11 8 8 & 11 8 8 & 11 8 8 & 10 8 & 10"
ValueError (not raised): label "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
No shield color found in this label: "8 8 <SOS> <SOS> 6 6 6 6 <SOS> 6 6 6 <SOS> 6 6 6 <SOS> 6 6"
ValueError (not rai

No shield color found in this label: "1 2 guard rampant rampant head head 6 6 6 6 6 6 6 6 6 6 6 6"
Caption Class - exception in label 1 2 guard rampant rampant head head 6 6 6 6 6 6 6 6 6 6 6 6, ['guard', 'rampant', 'rampant', 'head', 'head']
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> 6 6 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> 6 6 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> 6 6 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4 8 8 <SOS> 4 8 <SOS> 6 6 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 4

...

ValueError (not raised): label "1 1 8 8 8 8 <SOS> <SOS> s s s doubleheaded 8 4 8 8 4 8 8" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 8 8 8 8 <SOS> <SOS> s s s doubleheaded 8 4 8 8 4 8 8" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
Caption Class - exception in label 1 1 8 8 8 8 <SOS> <SOS> s s s doubleheaded 8 4 8 8 4 8 8, ['s']
ValueError (not raised): label "1 8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 8 8 <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS> <SOS>" cannot be parsed. The chunk "<SOS>" cann

No shield color found in this label: "1 1 2 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "8 8 2 9 9 9 9 9 9 4 4 8 9 4 4 8 8 9 9"
Caption Class - exception in label 1 1 1 1 4 9 9 9 9 9 s doubleheaded 4 9 9 s doubleheaded 9 s, ['doubleheaded', 'doubleheaded']
ValueError (not raised): label "1 1 2 guard a <SOS> patonce head 6 6 6 6 patonce doubleheaded 6 6 6 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
Caption Class - exception in label 1 1 2 guard a <SOS> patonce head 6 6 6 6 patonce doubleheaded 6 6 6 6 6, ['guard', '<SOS>', 'patonce', '6', 'patonce']
No shield color found in this label: "1 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
No shield color found in this label: "lion 9 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
No shield color found in this label: "11 1 1 8 8 8 8 8 8 9 10 8 9 10 8 9 10 8 9"
ValueError (not raised): label "1 1 1 <SOS> 4 8 8 4 8 8 & s s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any c

No shield color found in this label: "1 2 guard guard"
Caption Class - exception in label 1 2 guard guard, ['guard', 'guard']
Caption Class - exception in label s v 2 patonce, ['patonce']
ValueError (not raised): label "1 1 1 1 <SOS> 1 <SOS> 1 <SOS> 9 9 9 s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 1 1 <SOS> 1 <SOS> 1 <SOS> 9 9 9 s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 1 1 <SOS> 1 <SOS> 1 <SOS> 9 9 9 s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 1 8 8 8 <SOS> <SOS> <SOS> <SOS> <SOS> s s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 1 8 8 8 <SOS> <SOS> <SOS> <SOS> <SOS> s s s s s s s s" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "1 1 

No shield color found in this label: "1 1 lion 9 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
No shield color found in this label: "11 1 11 8 11 8 11 8 11 8 11 8 & 11 8 & 11 8 &"
Caption Class - exception in label o a 5 moline, ['moline']
No shield color found in this label: "1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1"
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> 6 6 6 6 6 6 6 6 6 6 6 6 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> 6 6 6 6 6 6 6 6 6 6 6 6 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
ValueError (not raised): label "8 8 <SOS> <SOS> <SOS> 6 6 6 6 6 6 6 6 6 6 6 6 6 6" cannot be parsed. The chunk "<SOS>" cannot be fit into any category.
No shield color found in this label: "8 8 <SOS> <SOS> <SOS> 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
Caption Class - exception in label g g 9 patonce, ['patonce']
No shield color found in this label: "1 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
No shield colo

([3.5938282012939453,
  3.5968122482299805,
  3.594947099685669,
  3.593860626220703,
  3.594775915145874,
  3.5929458141326904,
  3.599088430404663,
  3.5928757190704346,
  3.5902113914489746,
  3.596248149871826,
  3.5955121517181396,
  3.5948703289031982],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0

In [30]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)

## Evaluate a vanilla model on the synthetic test data - single


In [23]:


# data_location =  '../baseline-gen-data/large'
data_location =  '/home/space/datasets/COA/generated-data-api-single/'
# root_folder_images = data_location + '/images'
root_folder_images = data_location + '/res_images'
test_psumsq_file = data_location + '/test_captions_psumsq.txt'
df = pd.read_csv(test_psumsq_file)

print("There are {} test images".format(len(df)))

#Hyperparams
embed_size=300
vocab_size = 36
attention_dim=256
encoder_dim=512 #2048
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0
batch_size = 125
NUM_WORKER = 2
freq_threshold = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }


test_loader, test_dataset = init_testing_model(test_psumsq_file, root_folder_images, 
                                               mean, std,
                                               NUM_WORKER,vocab,batch_size, 
                                               device, pin_memory=False,
                                               img_h=500, img_w=500)


import torch
import random
import numpy as np
seed = 1234
random.seed(seed)     # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False




There are 144 test images


In [24]:

pretrained = False
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)

test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', real_data=False) 


No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError 

No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (no

No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label

No shield color found in this label: "doubleheaded 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category

No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
No shield color found in this label: "doubleheaded 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError 

...

ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 9 <PAD> <P

ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD>" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 

No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> s s s s s s s s s" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> s s s s s s s s s" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> s s s s s s s s s" cannot be parsed. The chunk "<PAD>" cannot be fit into any category.
ValueError (not raised): label "doubleheaded 9 9 9 9 <PAD> <PAD> <PAD> <PAD> <PAD> s s s s s s s s s" canno

No shield color found in this label: "doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9"
Caption Class - exception in label doubleheaded 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9, ['doubleheaded']
Test Accuracy (in progress): 0.0%
Test Accuracy (Overall): 0.0%
Test Loss (final):  3.8


([3.80014705657959, 3.809007406234741],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
 

In [26]:
pretrained = True
model, optimizer, criterion = get_new_model(hyper_params, learning_rate, ignored_idx, drop_prob, device, pretrained)

test_model(model, criterion,test_loader, test_dataset, vocab_size, device, 'vanilla-logs', real_data=False) 

ValueError (not raised): label "3 1 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 

No shield color found in this label: "6 6 6 6 6 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "3 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
ValueError (not raised): label "3 1 1 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 1 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 1 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 1 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category

No shield color found in this label: "6 6 6 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
ValueError (not raised): label "3 3 1 6 11 lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 6 11 lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 6 11 lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 6 11 lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 6 11 lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK> lion <UNK>" cannot be parsed. The chun

No shield color found in this label: "3 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "3 1 6 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
ValueError (not raised): label "3 3 1 3 1 <UNK> 3 <UNK> <UNK> <UNK> <UNK> & & & & & & v <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 3 1 <UNK> 3 <UNK> <UNK> <UNK> <UNK> & & & & & & v <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 3 1 <UNK> 3 <UNK> <UNK> <UNK> <UNK> & & & & & & v <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 3 1 <UNK> 3 <UNK> <UNK> <UNK> <UNK> & & & & & & v <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 3 1 3 1 <UNK> 3 <UNK> <UNK> <UNK> <UNK> & & & & & & v <UNK>" cannot be parsed. The chunk "<UNK>" cannot be fit into any ca

No shield color found in this label: "3 6 6 6 6 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
Test Accuracy (in progress): 0.0%
No shield color found in this label: "6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6"
No shield color found in this label: "3 6 6 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "3 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "3 1 1 1 1 3 11 11 11 3 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11"
No shield color found in this label: "6 6 6 6 6 6 6 6 11 11 11 11 11 11 11 11 11 11 11"
ValueError (not raised): label "3 1 6 6 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> & v &" cannot be parsed. The chunk "<UNK>" cannot be fit into any category.
ValueError (not raised): label "3 1 6 6 6 6 6 v <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> & v &" cannot be parsed. The chunk "<UNK>" cannot be fit into any cat

Test Accuracy (Overall): 1.0%
Test Loss (final):  3.6


([3.603231191635132, 3.6024892330169678],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.165,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.335,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0